In [3]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from keras.models import load_model

path = "assets/"
img_originale = cv2.imread(path + "formation_data_ia.jpeg")

import pickle
model = pickle.load(open("Thomas_CNN.h5","rb"))
# model = load_model('1472_CNN.h5')

In [6]:
def resize_image(img, size=(20,20)):

    h, w = img.shape[:2]
    
    if h == w: 
        return cv2.resize(img, size, cv2.INTER_AREA)

    dif = h if h > w else w


    if dif > (size[0] + size[1]):
        interpolation = cv2.INTER_AREA
    else:
        interpolation = cv2.INTER_CUBIC

    x_pos = (dif - w)//2
    y_pos = (dif - h)//2

    mask = np.zeros((dif, dif), dtype=img.dtype)
    mask[y_pos:y_pos+h, x_pos:x_pos+w] = img[:h, :w]

    return cv2.resize(mask, size, interpolation)

def evaluate(image_lettre):

    alphabet = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z', '<3']

    image_lettre = np.expand_dims(image_lettre, axis=(0,-1))
    prediction = np.argmax(model.predict(image_lettre) , axis=1)[0]
    lettre = alphabet[prediction]

    return lettre

## Mes tests de OpenCV

In [7]:
img_traite = img_originale.copy()

# imgBlur = cv2.GaussianBlur(img_traite, (9, 9), 1)
img_traite = cv2.cvtColor(img_traite, cv2.COLOR_BGR2GRAY)
imgCanny = cv2.Canny(img_traite, 80, 80)
kernel = np.ones((5, 5))
imgDil = cv2.dilate(imgCanny, kernel, iterations=1)
contours = cv2.findContours(imgDil, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours = contours[0] if len(contours) == 2 else contours[1]
contours = [cnt for cnt in contours if cv2.contourArea(cnt) > 500]

img_affichage = img_traite.copy()

for cnt in contours:
    x, y, w, h = cv2.boundingRect(cnt)

    image_lettre = img_affichage[y:y+h, x:x+w]

    image_lettre = cv2.bitwise_not(image_lettre)
    image_lettre = resize_image(image_lettre)
    image_lettre = cv2.copyMakeBorder(image_lettre, 4, 4, 4, 4, cv2.BORDER_CONSTANT)

    predictions = list()
    lettre = evaluate(image_lettre)

    cv2.putText(img_affichage, lettre, (x-10, y-10), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 0), 3)
    cv2.rectangle(img_affichage, (x , y ), (x + w , y + h ), (0, 0, 0), 5)

print(img_originale.shape)
print(img_traite.shape)
print(img_affichage.shape)

cv2.imshow("image", img_affichage)
cv2.waitKey()
cv2.destroyWindow("image")
del(img_traite, imgCanny, image_lettre, img_affichage)

(592, 1504, 3)
(592, 1504)
(592, 1504)


In [ ]:
im = cv2.imread(path + "gugu.png")
im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
im = cv2.GaussianBlur(im, (3,3), 0)
im = cv2.Canny(im,100,100)

kernel = np.ones((2,2),np.uint8)
im = cv2.dilate(im, kernel, iterations = 1)

kernel = np.ones((5,5),np.uint8)
imgEroded = cv2.erode(im, kernel, iterations= 5)

# cv2.imshow('image', im)
# cv2.waitKey(0)
# cv2.destroyWindow('image')

plt.imshow(im, cmap='gray')
print(im.shape)

## Version originale du tutoriel

In [4]:
def stackImages(scale,imgArray):
    rows = len(imgArray)
    cols = len(imgArray[0])
    rowsAvailable = isinstance(imgArray[0], list)
    width = imgArray[0][0].shape[1]
    height = imgArray[0][0].shape[0]
    if rowsAvailable:
        for x in range ( 0, rows):
            for y in range(0, cols):
                if imgArray[x][y].shape[:2] == imgArray[0][0].shape [:2]:
                    imgArray[x][y] = cv2.resize(imgArray[x][y], (0, 0), None, scale, scale)
                else:
                    imgArray[x][y] = cv2.resize(imgArray[x][y], (imgArray[0][0].shape[1], imgArray[0][0].shape[0]), None, scale, scale)
                if len(imgArray[x][y].shape) == 2: imgArray[x][y]= cv2.cvtColor( imgArray[x][y], cv2.COLOR_GRAY2BGR)
        imageBlank = np.zeros((height, width, 3), np.uint8)
        hor = [imageBlank]*rows
        hor_con = [imageBlank]*rows
        for x in range(0, rows):
            hor[x] = np.hstack(imgArray[x])
        ver = np.vstack(hor)
    else:
        for x in range(0, rows):
            if imgArray[x].shape[:2] == imgArray[0].shape[:2]:
                imgArray[x] = cv2.resize(imgArray[x], (0, 0), None, scale, scale)
            else:
                imgArray[x] = cv2.resize(imgArray[x], (imgArray[0].shape[1], imgArray[0].shape[0]), None,scale, scale)
            if len(imgArray[x].shape) == 2: imgArray[x] = cv2.cvtColor(imgArray[x], cv2.COLOR_GRAY2BGR)
        hor= np.hstack(imgArray)
        ver = hor
    return ver

In [5]:
def getContours(img,imgContour):
    a, b, c = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    contours = b

    # contours, hierarchy = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    
    for cnt in contours:
        area = cv2.contourArea(cnt)
        areaMin = cv2.getTrackbarPos("Area", "Parameters")
        if area > areaMin:
            cv2.drawContours(imgContour, cnt, -1, (255, 0, 255), 7)
            peri = cv2.arcLength(cnt, True)
            approx = cv2.approxPolyDP(cnt, 0.02 * peri, True)
            # print(len(approx))
            x , y , w, h = cv2.boundingRect(approx)
            cv2.rectangle(imgContour, (x , y ), (x + w , y + h ), (0, 255, 0), 5)

            cv2.putText(imgContour, "Points: " + str(len(approx)), (x + w + 20, y + 20), cv2.FONT_HERSHEY_COMPLEX, .7, (0, 255, 0), 2)
            cv2.putText(imgContour, "Area: " + str(int(area)), (x + w + 20, y + 45), cv2.FONT_HERSHEY_COMPLEX, 0.7, (0, 255, 0), 2)

def empty(a):
    pass

In [6]:
cv2.namedWindow("Parameters")
cv2.resizeWindow("Parameters",640,240)
cv2.createTrackbar("Threshold1","Parameters",80,255,empty)
cv2.createTrackbar("Threshold2","Parameters",80,255,empty)
cv2.createTrackbar("Area","Parameters",500,3000,empty)

img = cv2.imread(path + "formation_data_ia.jpeg")

while True:

    imgContour = img.copy()
    imgBlur = cv2.GaussianBlur(img, (9, 9), 1)
    imgGray = cv2.cvtColor(imgBlur, cv2.COLOR_BGR2GRAY)
    threshold1 = cv2.getTrackbarPos("Threshold1", "Parameters")
    threshold2 = cv2.getTrackbarPos("Threshold2", "Parameters")
    imgCanny = cv2.Canny(imgGray, threshold1, threshold2)
    kernel = np.ones((5, 5))
    imgDil = cv2.dilate(imgCanny, kernel, iterations=1)
    getContours(imgDil, imgContour)
    imgStack = stackImages(0.8, ([img,imgCanny], [imgDil,imgContour]) )

    cv2.imshow("Result", imgStack)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()